In [14]:
import numpy as np
import pandas as pd 
import  category_encoders as ce
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder

In [15]:
data_path = "../data/notebooks/4_merged_data.csv"

In [16]:
df_raw = pd.read_csv(data_path)
df = df_raw.copy()

C:\Users\saima\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
cols = ['launched_at',  'status', 'days_to_deadline', 'goal',
       'sub_category', 'category', 'blurb_length', 'location_country',  'rewards_mean', 'rewards_median',
       'rewards_variance', 'rewards_SD', 'rewards_MIN', 'rewards_MAX' ,
        'rewards_NUM', 'currency', 'launch_year', 'launch_month',
        'deadline_year', 'deadline_month']
target_encoding_cols = ['location_country' , 'currency' , 'category', 'sub_category']

train_years =[2016, 2017  , 2018 , 2019]


In [18]:
def pre_proc(df):
    df = df[cols]
    df= df.dropna(axis=0, subset=["rewards_MIN"])
    df= df.dropna(axis=0, subset=["blurb_length"])
    df.sort_values("launched_at" , inplace=True)
    df = df.reset_index(drop=True)
    df["launched_at"]  = pd.to_datetime(df["launched_at"]).dt.date
    df.sort_values("launched_at" , inplace=True)
    df.drop(['launched_at', ] ,axis=1 , inplace=True)
    df.reset_index(inplace=True)
    df.drop('index', inplace=True , axis=1)
    
    binarizer= LabelBinarizer()
    df["status"] = binarizer.fit_transform(df["status"])
    
    
    return df , binarizer


    

In [19]:
def onehot_categ(df):
    
    encoder  = OneHotEncoder(sparse=False)
    X_hot = encoder.fit_transform(df[ target_encoding_cols])
    
    onehotcols = []
    for cat in encoder.categories_:
        for col in cat:
            onehotcols.append(col)
            
    X_hot = pd.DataFrame(X_hot , columns=onehotcols)
    df =pd.concat([df , X_hot] , axis=1)
    df.drop(target_encoding_cols , axis=1 , inplace=True)
    
    
    return df , encoder
    

In [20]:
def get_model_data(df , train_years ):
    
    df_train = df[df['launch_year'].apply(lambda x: True if x in train_years else False)]
    X_train , y_train = df_train.drop("status" , axis=1) , df_train['status']
    
    return X_train , y_train

In [21]:
def helmert_categ(df_train):
    encoder = ce.HelmertEncoder(cols = target_encoding_cols , drop_invariant=True )
    dfh = encoder.fit_transform(df_train[target_encoding_cols])
    df_train = pd.concat([df_train , dfh], axis=1)
    df_train.drop(target_encoding_cols , axis=1 , inplace=True)
    
    return df_train , encoder

In [22]:
from xgboost import XGBClassifier
import operator

def XG_train(X_train,  y_train):
    
    XG= XGBClassifier(n_estimators=150, random_state=3)
    XG.fit(X_train, y_train)
   
    return XG


In [23]:
df_proc , binarizer  = pre_proc(df)

In [24]:

df_onehot ,onehotencoder  = onehot_categ(df_proc)
X_train_oh , y_train_oh= get_model_data(df_onehot , train_years )

In [25]:
X_train_raw , y_train_hel  = get_model_data(df_proc , train_years)
X_train_hel ,helmert_encoder  = helmert_categ(X_train_raw)


In [26]:
model_oh = XG_train(X_train_oh ,y_train_oh)

In [27]:
model_hel = XG_train(X_train_hel ,y_train_hel)

### Saving models and encoders

In [28]:

from sklearn.externals import joblib
def save_estimator(model , path):
    try:
        joblib.dump(model , path)
    except Exception as e:
        return e
        
        
    return "Saved sucessfully"

C:\Users\saima\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [39]:
print(save_estimator(model_oh , 'estimators/model_oh.sav'))
print(save_estimator(model_hel , 'estimators/model_hel.sav'))

print(save_estimator(onehotencoder  , 'estimators/encoder_oh.sav'))
print(save_estimator(helmert_encoder, 'estimators/encoder_hel.sav'))

print(save_estimator(binarizer , 'estimators/encoder_label.sav'))

Saved sucessfully
Saved sucessfully
Saved sucessfully
Saved sucessfully
Saved sucessfully
